In [ ]:
from utils.imports import *
from utils.start_spark import spark
! start http://localhost:4040/jobs/

Uma forma de transformar a target é, em vês da planeada "dataframe de sessões", teriamos uma "dataframe de perguntas", onde em vês de estar uma dataframe com cada linah uma sessão, estar 18 das mesmas linhas, com a diferença nas questões. Ou seja, a nossa chave primária seria o conjunto da das colunas `session_id` e `question` (categórica ou inteira), em vês de apenas ter como chave primária `session_id`.

| session_id | q1 | q2 | ... | q18 | feature1 | feature2 | ... |
|------------|----|----|-----|-----|----------|----------|-----|
| 1          | 0  | 1  | ... | 1   | 423      | 0        | ... |
| 2          | 1  | 0  | ... | 1   | 231      | 1        | ... |
| 3          | 1  | 1  | ... | 1   | 345      | 1        | ... |

para

| session_id | question | answer | feature1 | feature2 | ... |
|------------|----------|--------|----------|----------|-----|
| 1          | 1        | 0      | 423      | 0        | ... |
| 1          | 2        | 1      | 423      | 0        | ... |
| ...        | ...      | ...    | ...      | ...      | ... |
| 1          | 18       | 1      | 423      | 0        | ... |
| 2          | 1        | 1      | 231      | 1        |     |
| ...        | ...      | ...    | ...      | ...      | ... |

Esta opção iria facilitar o processo de previsão, pois podemos usar métodos simples como regressão logística, usando `question` como uma feature também.

Uma segunda opção seria usar classificação *multi-label*, sendo que temos vários `targets`, mas estes só têm uma carnidalidade de 2. Isto iria aumentar a complexidade do projeto. Estamos também limitados pela utilização de MLLIB do spark.

A terceira opção seria treinar 18 modelos, e criar um algoritmo para escolher um modelos na precisão. Essencialmente, este é o método de *binary relevance*, sendo um dos métodos da segunda opção; no entanto, estamos a notar como opção porque não parece que o MLLIB tenha qualquer opção de *binary relevance*.

Nota: da forma como os dados estão agora formatados, essencialmente é o mesmo problema se tivéssemos em vês de uma matriz de 0s e 1s, uma coluna com um array das classes onde a sessão teve certo. 

Decidimos usar *binary relevance*, ou seja, vamos treinar um modelo para cada pergunta.

Para análise das métricas, vamos usar `MultilabelClassificationEvaluator` do MLlib.

In [ ]:
doing_features = False
training = False
timing = False
evaluate = False

In [ ]:
if 'stages' in sys.modules: del sys.modules['stages']
from stages import *

pipeline_no_ML = Pipeline(stages = [
    the_transformer(
        add_id,
        (elapsed_to_diff, "elapsed_time", "elapsed_diff_ms"),
        (negative_to_0, "elapsed_diff_ms"),
        (elapsed_to_hours, "elapsed_time", "elapsed_time_h"),
        agg1,
        agg2,
        typeOfText,
    ),
    VectorAssembler(inputCols = ["inv_total_time_h", "inv_total_time_h_0-4", "inv_total_time_h_5-12"], outputCol = "inv_time_to_stand"),
    StandardScaler(inputCol = "inv_time_to_stand", outputCol = "inv_time_standed", withMean = True),
    StringIndexer(inputCol = "type_of_script", outputCol = "index_of_type_of_script"),
    OneHotEncoder(inputCol = "index_of_type_of_script", outputCol="dummies_of_type_of_script"), #sparse vector
])

In [ ]:
if doing_features:
    train = spark.read.parquet(".\\data\\df_train\\")
    test = spark.read.parquet(".\\data\\df_test\\")

    transformer = pipeline_no_ML.fit(train)
    trans_train = transformer.transform(train)
    # pivoted {
    from files.dfs import train_labels as labels
    if labels._isRead == False: labels.read(spark)
    splited = labels.df \
        .select(
            split("session_id", "_").alias("both"),
            "correct"
        ).select(
            col("both")[0].alias("session_id"),
            col("both")[1].alias("question"),
            col("correct").alias("isCorrect")
        )
    pivoted = splited \
        .groupby("session_id") \
        .pivot("question") \
        .agg(first("isCorrect"))
    # }
    # add pivoted
    trans_joined = trans_train.join(pivoted, ["session_id"], "left")
    trans_joined.drop(trans_joined.session_id)
    trans_joined.write.mode("overwrite").parquet(r"data\trans_train")

    trans_test = transformer.transform(test)
    trans_test.drop(trans_test.session_id)
    trans_test.write.mode("overwrite").parquet(r"data\trans_test")

    trans_train = trans_joined
    
    train = trans_train
    test = trans_test

In [ ]:
if not doing_features:
    train = spark.read.parquet(r"data\trans_train")
    test = spark.read.parquet(r"data\trans_train")

In [ ]:
features_ass = VectorAssembler(inputCols = [
 'max_index',
 'obs_opcional',
 'obs_no_in',
 'notebook_opens',
 'notebook_explorer',
 'fullscreen',
 'hq',
 'music',
 'avg_elapsed_diff_ms_cutscene',
 'avg_elapsed_diff_ms_person',
 'avg_elapsed_diff_ms_navigate',
 'inv_time_standed',
 'dummies_of_type_of_script'
], outputCol="features")

In [ ]:
if 'utils.models' in sys.modules: del sys.modules['utils.models']
from utils.models import Model
LR = Model(LogisticRegression, train, test, features_ass, name = "lr")
SVM = Model(LinearSVC, train, test, features_ass, name= "svm")
DT = Model(DecisionTreeClassifier, train, test, features_ass, name="dt")
RF = Model(RandomForestClassifier, train, test, features_ass, name="rf")

models1:list[Model] = [LR, SVM, DT, RF]

In [ ]:
if training:
    for i in models1:
        i.train_and_predict(write = True)

In [ ]:
if not training:
    for i in models1:
        i.load_tested(spark)

In [ ]:
class timeit():
    from datetime import datetime
    def __enter__(self):
        self.tic = self.datetime.now()
    def __exit__(self, *args, **kwargs):
        print('runtime: {}'.format(self.datetime.now() - self.tic))

In [ ]:
if timing:
    for i in models1:
        print(i.name)
        for a in range(5):
            with timeit():
                i.train_and_predict(write=False)

In [ ]:
if evaluate:
    for i in models1:
        print(i.evaluate())

In [ ]:
RF_all_strat = Model(RandomForestClassifier, train, test, features_ass, name="rf_all_strat", featureSubsetStrategy = "all")
RF_sqrt_strat = Model(RandomForestClassifier, train, test, features_ass, name="rf_sqrt_strat", featureSubsetStrategy = "sqrt")
RF_log2_strat = Model(RandomForestClassifier, train, test, features_ass, name="rf_log2_strat", featureSubsetStrategy = "log2")
RF_onethird_strat = Model(RandomForestClassifier, train, test, features_ass, name="rf_onethird_strat", featureSubsetStrategy = "onethird")

models2 = [RF_all_strat, RF_sqrt_strat, RF_log2_strat, RF_onethird_strat]


In [ ]:
if training:
    for i in models2:
        i.train_and_predict(write = True)

In [ ]:
if not training:
    for i in models2:
        i.load_tested(spark)

In [ ]:
if evaluate:
    for i in models2:
        print(i.evaluate())

In [ ]:
# confusion matrix
from sklearn.metrics import multilabel_confusion_matrix as cm
from sklearn.preprocessing import MultiLabelBinarizer
RF_onethird_strat.evaluate()
forEvalP = RF_onethird_strat.forEval.toPandas()
mlb = MultiLabelBinarizer().fit_transform

In [ ]:
cm(y_true = mlb(forEvalP["label"]), y_pred = mlb(forEvalP["prediction"]), samplewise=False)

array([[[  573,  4476],
        [  288, 13366]],

       [[    0,   392],
        [    0, 18311]],

       [[   10,  1190],
        [    0, 17503]],

       [[  414,  3332],
        [  123, 14834]],

       [[ 3918,  4485],
        [ 2240,  8060]],

       [[  257,  3899],
        [  103, 14444]],

       [[  255,  4613],
        [  122, 13713]],

       [[  580,  6557],
        [  293, 11273]],

       [[  249,  4617],
        [  106, 13731]],

       [[ 5237,  3982],
        [ 3097,  6387]],

       [[  840,  5759],
        [  471, 11633]],

       [[    0,  2520],
        [    0, 16183]],

       [[13404,   120],
        [ 4946,   233]],

       [[  402,  5053],
        [  174, 13074]],

       [[ 5465,  4208],
        [ 3125,  5905]],

       [[    0,  4958],
        [    0, 13745]],

       [[    6,  5785],
        [    0, 12912]],

       [[    1,   930],
        [    0, 17772]]], dtype=int64)